# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
#notice this import looks different. Helpful documentation: http://statsmodels.sourceforge.net/devel/importpaths.html
#importing statsmodels.api will load most of the public parts of statsmodels. 
#This makes most functions and classes conveniently available within one or two levels, without making the “sm” namespace too crowded.
import pylab as pl
import numpy as np


In [4]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [5]:
# frequency table for prestige and whether or not someone was admitted
#pd.crosstab([df['admit'],df['prestige']],[df['gpa']])

pd.crosstab(df['admit'],[df['prestige']])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


Explanation: In the frequency table we are are basically counting the number of observations on the unique columns of the Dataframe.  In this case we have an admit column and a prestige column and both are categorical variables as we verified in the previous unit projects. For each admit category AND each prestige category, we want to tabulate the number of observations for each combination of categories.  
I will do this using Pandas "crosstab" function.  There is an 'index' argument and a 'column' argument. The index argument says "values to group-by in the rows". The rows refer to the tabulated table rows (admit in the example above). We want admit to be the rows, so I can pass in the admit column of the Dataframe like this: df['admit']. The crosstab function will know to group all the 1 observations together and the 0 observations together. Then in the column argument to crosstab,I can pass in the prestige data. Crosstab will take the groups of admit = 1 and 0 and further segment those groups by prestige levels and just do a simple count of how many obeservations are in each.

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [6]:
dummy_prestige = pd.get_dummies(df['prestige'],prefix='prestige')
print dummy_prestige

     prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0               0             0             1             0
1               0             0             1             0
2               1             0             0             0
3               0             0             0             1
4               0             0             0             1
5               0             1             0             0
6               1             0             0             0
7               0             1             0             0
8               0             0             1             0
9               0             1             0             0
10              0             0             0             1
11              1             0             0             0
12              1             0             0             0
13              0             1             0             0
14              1             0             0             0
15              0             0         

#### 2.2 When modeling our class variables, how many do we need? 



Answer: Since only prestige is the only categorical variable (GRE is discrete and GPA is continuous), we only need dummy variables for prestige.  Each dummy variable column is dichotomous and there are four ranks (1-4).  When creating dummy variables we set a baseline, therefore we need three additional dummy_presitige variables.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [7]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_prestige.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [8]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc['admit'],[handCalc['prestige_1.0']])

prestige_1.0,0,1
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [9]:
#P(admitted in prestige_1.0)

prob = float(33)/61*100
print prob

odds1 = float(33)/28
print odds1

54.0983606557
1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [10]:
oddsOther = float(93)/243
print oddsOther

0.382716049383


In [11]:
#odds1 = float(33)/28
#odds2 = float(53)/95
#odds3 = float(28)/93
#odds4 = float(12)/55

#### 3.3 Calculate the odds ratio

In [12]:
OR = odds1/oddsOther
print OR

3.07949308756


#### 3.4 Write this finding in a sentenance: 

Answer: The odds ratio here tells us that the odds of admission are approximately 300% higher (Odds Ratio x 100%) for students who went to a prestige_1 school than they are for students who did not go to a prestige_1 school.

#### 3.5 Print the cross tab for prestige_4

In [13]:
pd.crosstab(handCalc['admit'],[handCalc['prestige_4.0']])

prestige_4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [14]:
odds4 = float(12)/55
oddsOthers = float(114)/216
OR = odds4/oddsOthers
print OR

0.413397129187


#### 3.7 Write this finding in a sentence

Answer: The odds ratio here tells us that the odds of admission are approximately 60% lower for students who went to a prestige_4 school than they are for students who did not go to a prestige_4 school.

## Part 4. Analysis

In [15]:
# create a clean data frame for the regression
# why do we only have prestige_2, prestige_3, and prestige_4
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_prestige.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [16]:
# manually add the intercept
data['intercept'] = 1.0

Explanation:  Why does adding an intercept column of all 1's actually represent an intercept term in the regression?  Because if one of the predictors is always 1.0, its Beta value will be one (i.e. no change on overall results), and so it effectively just adds 1 to the regression = the intercept.

#### 4.1 Set the covariates to a variable called train_cols

In [17]:
train_cols = ['gre','gpa','prestige_2.0','prestige_3.0','prestige_4.0','intercept']
print train_cols

#This is essentially our X values
#I dont understand why the intercept is a covariate

['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0', 'intercept']


#### 4.2 Fit the model

In [20]:
#run a logistic regression using statsmodels
#MLE = maximum likelihood estimate
LLogit = sm.Logit(data['admit'],data[train_cols])
FFit = LLogit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


Explanation: Model fitting is a procedure that can be summarized in three steps...
(1) First you need a function that takes in a set of parameters and returns a predicted data set.
(2) Second you need an 'error function' that provides a number representing the difference between your data and the model's prediction for any given set of model parameters. This is usually either the sums of squared error (SSE) or maximum likelihood.
(3) Third you need to find the parameters that minimize this difference. Once you set things up properly, this third step is easy thanks to the nerds at Mathworks.

The fit process involves two steps...
(1) Create a Logit object passing in the correct arguments. "endog" is the dependent variable according to the documentation while "exog" is the independent variables. 
(2) Fit the model. Does the fit model need any arguments? Check out the documentation for the fit method.
Documentation: http://statsmodels.sourceforge.net/0.6.0/generated/statsmodels.discrete.discrete_model.Logit.html

#### 4.3 Print the summary results

In [19]:
summ = FFit.summary()
print summ

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sat, 31 Dec 2016   Pseudo R-squ.:                 0.08166
Time:                        13:11:20   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [ ]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [ ]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

#### 5.1 Recreate the dummy variables

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.